# Виконання

## Основне завдання

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
df = pd.read_csv('bbc-news-data.csv', sep='\t')
df

*Зчитування файлу*

### Видалимо колонки 'filename', 'title', 'category'.

In [ ]:
df.drop(['filename', 'title', 'category'], axis=1, inplace=True)
df

*Видалення колонок*

### Видалимо порожні документи, якщо вони є.

In [ ]:
df = df[~(df.content.str.strip() == '')]
df

*Видалення порожніх документів*

### Визначимо стоп-слова англійської мови.

In [ ]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

*Стоп-слова*

### Визначимо функцію, що виконує попередню обробку документу. Застосуємо декоратор np.vectorize для того, щоб функція могла працювати з корпусами.

In [ ]:
@np.vectorize
def preproc_doc(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()
    tokens = wpt.tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens)
    return doc

p_corpus = preproc_doc(df.content)
p_corpus

*Обробка документів*

### Розіб'ємо кожний документ на окремі слова, об'єднаємо усі слова в одну сукупність.

In [ ]:
words = []
for doc in p_corpus:
    words.extend(doc.split(' '))
df_words = pd.DataFrame(set(words))
df_words.columns = ['words']
df_words.head(10)

*Розбиття на слова*

### Визначимо частину мови для кожного слова. Використаємо функцію nltk.pos_tag.

In [ ]:
df_words['ps'] = [tag for _, tag in nltk.pos_tag(df_words.words)]
df_words

*Визначення частини мови*

### Узагальнимо частини мови до звичайних: noun, adective, verb тощо.

In [ ]:
from nltk.tag import map_tag
df_words['sps'] = [map_tag('en-ptb', 'universal', tag) for tag in df_words.ps]
df_words

*Узагальнення чатин мов*

### Перетворимо узагальнені частини мови на абревіатури для лематизації.

In [ ]:
abbr = {'NOUN': 'n', 'VERB': 'v', 'ADJ': 'a', 'ADV': 'r'}
def to_abbr(el):
    res = abbr.get(el)
    if res is not None:
        return res
    return ''

df_words['abbr'] = [to_abbr(x) for x in df_words.sps]
df_words

*Приведення загальних частин мов до абревіатур*

### Проведемо лематизацію кожного слова за допомогою методу lemmatize об'єкта класу nltk.stem.WordNetLemmatizer.

In [ ]:
from nltk.stem import WordNetLemmatizer
wlem = WordNetLemmatizer()
def my_lem(word, abbr):
    if abbr == '':
        return wlem.lemmatize(word)
    return wlem.lemmatize(word, pos=abbr)
df_words['lemms'] = [my_lem(row[1]['words'], row[1]['abbr']) 
               for row in df_words.loc[:, ['words', 'abbr']].iterrows()]
df_words

*Лематизація слів*

### Представимо корпус як модуль "Сумка слів". Використаємо для цього клас CountVectorizer зі sklearn.feature_extraction.text.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(p_corpus)
cv_matrix = pd.DataFrame(cv_matrix.toarray(), 
                         columns=cv.get_feature_names_out())
cv_matrix.to_csv('bag_of_words.csv')

*Сумка слів*

### Представимо корпус як модель TD-IDF. Перетворимо матрицю з частотою термінів на матрицю tfidf.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tt = TfidfTransformer(norm='l2', use_idf=True)
tt_matrix = tt.fit_transform(cv_matrix)
tt_matrix = tt_matrix.toarray()
vocab = cv.get_feature_names_out()
tv_matrix = pd.DataFrame(np.round(tt_matrix, 2), columns=vocab)
tv_matrix

*Матриця TF-IDF*

### Підрахуємо частоту кожного слова.

In [ ]:
df_counts = pd.DataFrame(nltk.FreqDist(words).items(), columns=['words', 'counts'])
df_counts.set_index('words', inplace=True)
df_counts

*Частоти слів*

### Відсортуємо датафрейм за спаданням частоти.

In [ ]:
df_counts.sort_values(by='counts', ascending=False, inplace=True)
df_counts

*Сортований датафрейм*

### Зобразимо перші десять найбільш уживаних слів.

In [ ]:
df_counts.head(10)

*Найуживаніші слова*

### Виведемо метрику для перших десяти елементів

In [ ]:
tv_matrix[df_counts.index[:10].to_list()]

*Метрики для найбільш уживаних слів*

## Додаткове завдання 1

## Зчитаємо текст.

In [ ]:
df_ukr = pd.read_csv('ukr_text.csv')
df_ukr

*Зчитування файлу*

### Видалимо колонки 'id'.

In [ ]:
df_ukr.drop(['Id'], axis=1, inplace=True)
df_ukr

*Видалення колонок*

### Видалимо порожні документи, якщо вони є.

In [ ]:
df_ukr = df_ukr[~(df.content.str.strip() == '')]
df_ukr

*Видалення порожніх документів*

### Визначимо стоп-слова української мови. Завантажимо їх.

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/olegdubetcky/Ukrainian-Stopwords/main/ukrainian'
r = requests.get(url)
with open(nltk.data.path[0]+'/corpora/stopwords/ukrainian', 'wb') as f:
    f.write(r.content)

*Завантаження стоп-слів*

### Визначимо стоп-слова.

In [ ]:
import string
import pymorphy2
from nltk.corpus import stopwords
stop_words = stopwords.words("ukrainian")
morph = pymorphy2.MorphAnalyzer(lang='uk')
stop_words = pd.Series(list(set(stop_words+list(string.punctuation))))
stop_words

*Визначення стоп-слів*

In [ ]:
p_corpus_ukr = preproc_doc(df_ukr.Body)
p_corpus_ukr

*Список стоп-слів української мови*

### Створимо корпус слів.

In [ ]:
p_corpus_ukr = preproc_doc(df_ukr.Body)
p_corpus_ukr

*Обробка документів*